# LangGraph Workflow Testing

This notebook tests the new LangGraph workflow implementation.

In [7]:
# Setup imports
import sys
from pathlib import Path
from datetime import datetime, timedelta

# Add src to path
sys.path.insert(0, str(Path.cwd() / "src"))

# Import specific components from workflow
from agents.workflow import (
    WorkflowState,
    QuerySpec,
    UserProfile,
    Candidate,
    ExtractedEvent,
    RecommendedEvent,
    Decision,
    create_workflow,
    run_workflow,
    test_workflow
)

print("✅ Workflow module imported successfully!")

✅ Workflow module imported successfully!


## 1. Test Basic Workflow

In [2]:
# Run the workflow with a test query
result = run_workflow(
    query="Find protests and cultural events in NYC",
    city="NYC",
    days_ahead=14
)

print("Workflow executed successfully!")
print(f"\nQuery: {result['query_spec']['text']}")
print(f"City: {result['query_spec']['city']}")
print(f"Date Range: {result['query_spec']['date_from'].date()} to {result['query_spec']['date_to'].date()}")
print(f"\nCycles executed: {result['cycle_count']}")
print(f"Total cost: ${result['total_cost']:.4f}")
print(f"Events found: {len(result['top10'])}")

Loaded 20 test events
Searching Tavily with query: Find protests and cultural events in NYC NYC next 14 days 2025...
Found 15 results from Tavily
Workflow executed successfully!

Query: Find protests and cultural events in NYC
City: NYC
Date Range: 2025-08-18 to 2025-09-01

Cycles executed: 0
Total cost: $0.0300
Events found: 5


## 2. View Recommendations

In [3]:
# Show the top recommendations
print("Top Recommendations:")
print("=" * 50)

for i, event in enumerate(result['top10'], 1):
    print(f"\n{i}. {event['title']}")
    print(f"   Score: {event['recommendation']:.2f}")
    print(f"   Location: {event['location']}")
    print(f"   Access: {event['access_req']}")
    print(f"   Time: {event['time']}")
    print(f"   Rationale: {event['rationale']}")

Top Recommendations:

1. People rally during "No Kings" protests on June 14, 2025. Marches ...
   Score: 0.80
   Location: NYC
   Access: open to all
   Time: 2025-08-25 12:06:52.078414
   Rationale: Matches your interest in Cultural Event. Located in NYC. 

2. ‍  Pride 2025 This year's theme, “Rise Up: Pride in Protest,” is a ...
   Score: 0.70
   Location: NYC
   Access: open to all
   Time: 2025-08-25 12:06:52.078401
   Rationale: Matches your interest in Cultural Event. Located in NYC. 

3. That's a wrap on Fleet Week New York 2025 ... - Instagram
   Score: 0.70
   Location: NYC
   Access: open to all
   Time: 2025-08-25 12:06:52.078410
   Rationale: Matches your interest in Cultural Event. Located in NYC. 

4. Nationwide Protest Against Current Administration on June 14th, 2025
   Score: 0.70
   Location: NYC
   Access: open to all
   Time: 2025-08-25 12:06:52.078412
   Rationale: Matches your interest in Cultural Event. Located in NYC. 

5. We will not be erased. Tomorrow, join a

## 3. Examine Workflow Execution

In [4]:
# Show the workflow execution logs
print("Workflow Execution Logs:")
print("=" * 50)

for log in result['logs']:
    print(f"  • {log}")

Workflow Execution Logs:
  • Profile & Planner: Building search profile from test data
  • Profile & Planner: Built profile with 22 domains and 21 keywords
  • Profile & Planner: Search strategy uses 7 keywords
  • Retriever: Searching for events with Tavily API
  • Retriever: Found 7 candidates from Tavily
  • Extractor: Processing 7 candidates
  • Recommender: Scoring 5 events
  • Recommender: Decision=accept


## 4. User Profile Analysis

In [8]:
# Test the retriever's processing without making API calls
# Using mock data that mimics Tavily's response

mock_tavily_results = [
    {
        "url": "https://www.timeout.com/newyork/protests",
        "title": "March for Climate Action - Bryant Park April 2025",
        "content": "Join thousands for a peaceful march starting at Bryant Park on April 5, 2025. The protest aims to raise awareness about climate change...",
        "score": 0.75
    },
    {
        "url": "https://www.metmuseum.org/events",
        "title": "Met Gala 2025 - Fashion's Biggest Night",
        "content": "The Metropolitan Museum of Art announces the 2025 Met Gala scheduled for May. This exclusive cultural event brings together...",
        "score": 0.82
    },
    {
        "url": "https://www.nyc.gov/office-of-the-mayor",
        "title": "NYC Mayor's Office Cultural Events Initiative",
        "content": "Mayor announces funding for summer 2025 cultural festivals across all five boroughs. Events will run from June through August...",
        "score": 0.45
    }
]

# Process with date filtering
print("TESTING DATE FILTER (No API call)")
print("=" * 60)

date_from = datetime.now()
date_to = datetime.now() + timedelta(days=30)

# Mock the date filtering
filtered = []
for result in mock_tavily_results:
    if "April" in result["content"] or "May" in result["content"]:
        filtered.append(result)
        print(f"✓ KEPT: {result['title'][:50]}")
    else:
        print(f"✗ FILTERED: {result['title'][:50]} (outside date range)")

print(f"\nFiltered: {len(filtered)}/{len(mock_tavily_results)} results kept")

# Test re-ranking
print("\n" + "=" * 60)
print("TESTING RE-RANKING (No API call)")
print("=" * 60)

interests = ["protests", "cultural events", "fashion"]
keywords = ["climate", "gala", "march", "bryant park"]

for result in filtered:
    original_score = result["score"]
    new_score = original_score
    
    # Apply boosts
    for interest in interests:
        if interest.lower() in result["title"].lower() or interest.lower() in result["content"].lower():
            new_score += 0.1
    
    for keyword in keywords:
        if keyword.lower() in result["title"].lower() or keyword.lower() in result["content"].lower():
            new_score += 0.05
    
    result["relevance_score"] = min(new_score, 1.0)
    
    print(f"\n{result['title'][:50]}")
    print(f"  Original: {original_score:.2f} → Relevance: {result['relevance_score']:.2f}")
    if result["relevance_score"] > original_score:
        print(f"  Boosted by: +{result['relevance_score'] - original_score:.2f}")

TESTING DATE FILTER (No API call)
✓ KEPT: March for Climate Action - Bryant Park April 2025
✓ KEPT: Met Gala 2025 - Fashion's Biggest Night
✓ KEPT: NYC Mayor's Office Cultural Events Initiative

Filtered: 3/3 results kept

TESTING RE-RANKING (No API call)

March for Climate Action - Bryant Park April 2025
  Original: 0.75 → Relevance: 0.90
  Boosted by: +0.15

Met Gala 2025 - Fashion's Biggest Night
  Original: 0.82 → Relevance: 0.97
  Boosted by: +0.15

NYC Mayor's Office Cultural Events Initiative
  Original: 0.45 → Relevance: 0.55
  Boosted by: +0.10


### 9.4 Testing with Mock Data (No API Credits)

In [9]:
# Show how the Retriever processes Tavily results

# Convert Tavily results to our candidate format
def show_processing_pipeline():
    print("=" * 60)
    print("RETRIEVER PROCESSING PIPELINE")
    print("=" * 60)
    
    print("\n1. RAW TAVILY RESULTS → CANDIDATES")
    print("   For each result from Tavily:")
    print("   • Extract url, title, content (snippet), score")
    print("   • Truncate content to 500 characters")
    print("   • Sort by score (descending)")
    
    print("\n2. DATE FILTERING")
    print("   Check each candidate's snippet for:")
    print("   • Month names in date range (March, April, etc.)")
    print("   • Temporal keywords (next week, upcoming, scheduled)")
    print("   • Year mentions (2025)")
    print("   → Keep only candidates with relevant dates")
    
    print("\n3. RE-RANKING BY RELEVANCE")
    print("   Boost scores based on:")
    print("   • User interests match (+0.1 per interest)")
    print("   • Keyword match (+0.05 per keyword)")
    print("   • Final score capped at 1.0")
    print("   → Re-sort by new relevance scores")
    
    print("\n4. FINAL OUTPUT")
    print("   Return list of candidates with:")
    print("   • url, title, snippet")
    print("   • score (original Tavily score)")
    print("   • relevance_score (after our re-ranking)")

show_processing_pipeline()

# Example of score boosting
print("\n" + "=" * 60)
print("EXAMPLE: SCORE BOOSTING")
print("=" * 60)

example_candidate = {
    "title": "Climate Protest at Bryant Park",
    "snippet": "Major protest scheduled for next week...",
    "score": 0.5  # Original Tavily score
}

interests = ["protests", "cultural events", "political"]
keywords = ["protest", "march", "bryant park", "climate"]

print(f"\nOriginal score: {example_candidate['score']}")
print(f"Title: {example_candidate['title']}")
print(f"\nBoosts applied:")

relevance = example_candidate['score']
for interest in interests:
    if interest.lower() in example_candidate['title'].lower():
        print(f"  +0.1 for interest '{interest}'")
        relevance += 0.1

for keyword in keywords:
    if keyword.lower() in example_candidate['title'].lower():
        print(f"  +0.05 for keyword '{keyword}'")
        relevance += 0.05

print(f"\nFinal relevance score: {min(relevance, 1.0):.2f}")

RETRIEVER PROCESSING PIPELINE

1. RAW TAVILY RESULTS → CANDIDATES
   For each result from Tavily:
   • Extract url, title, content (snippet), score
   • Truncate content to 500 characters
   • Sort by score (descending)

2. DATE FILTERING
   Check each candidate's snippet for:
   • Month names in date range (March, April, etc.)
   • Temporal keywords (next week, upcoming, scheduled)
   • Year mentions (2025)
   → Keep only candidates with relevant dates

3. RE-RANKING BY RELEVANCE
   Boost scores based on:
   • User interests match (+0.1 per interest)
   • Keyword match (+0.05 per keyword)
   • Final score capped at 1.0
   → Re-sort by new relevance scores

4. FINAL OUTPUT
   Return list of candidates with:
   • url, title, snippet
   • score (original Tavily score)
   • relevance_score (after our re-ranking)

EXAMPLE: SCORE BOOSTING

Original score: 0.5
Title: Climate Protest at Bryant Park

Boosts applied:
  +0.05 for keyword 'protest'
  +0.05 for keyword 'bryant park'
  +0.05 for ke

### 9.3 Post-Processing Applied by Retriever

In [10]:
# Example of what Tavily API returns (based on actual response structure)
# This is NOT a real API call - just showing the structure

example_tavily_response = {
    "results": [
        {
            "url": "https://www.timeout.com/newyork/events-calendar/april-events-calendar",
            "title": "NYC events in April 2025 - Time Out",
            "content": "The best NYC events in April include much-needed outdoor activities, new exhibits, impressive theater, and pretty flower shows...",
            "score": 0.586,
            "published_date": None,
            "author": None,
            "raw_content": None  # Would contain full HTML if include_raw_content=True
        },
        {
            "url": "https://www.facebook.com/groups/50501movement/posts/1026118825792113/",
            "title": "Will be at the protest tomorrow Union Square ,New York City 12-3",
            "content": "Protest event organized by 50501 Movement at Union Square...",
            "score": 0.512,
            "published_date": "2025-03-15",
            "author": "50501 Movement",
            "raw_content": None
        },
        {
            "url": "https://www.nyc.gov/events",
            "title": "Welcome to NYC.gov | City of New York",
            "content": "Mayor Adams Announces Upcoming Cultural Events as Part of 'Founded by NYC' initiative...",
            "score": 0.457,
            "published_date": None,
            "author": None,
            "raw_content": None
        }
    ],
    "query": "Find cultural events and protests in NYC NYC next 14 days 2025",
    "follow_up_questions": None,
    "answer": None,  # Would contain AI-generated answer if asked for
    "images": [],    # Would contain images if include_images=True
    "results_count": 3
}

print("=" * 60)
print("EXAMPLE TAVILY API RESPONSE STRUCTURE")
print("=" * 60)

print("\n1. TOP-LEVEL FIELDS:")
for key in example_tavily_response.keys():
    value_type = type(example_tavily_response[key]).__name__
    print(f"   • {key}: {value_type}")

print("\n2. RESULT OBJECT STRUCTURE:")
if example_tavily_response["results"]:
    first_result = example_tavily_response["results"][0]
    for key, value in first_result.items():
        value_preview = str(value)[:50] + "..." if value and len(str(value)) > 50 else value
        print(f"   • {key}: {value_preview}")

print("\n3. WHAT WE EXTRACT FROM EACH RESULT:")
print("   • url → candidate['url']")
print("   • title → candidate['title']")  
print("   • content → candidate['snippet'] (truncated to 500 chars)")
print("   • score → candidate['score']")
print("   • published_date → candidate['published_date']")

EXAMPLE TAVILY API RESPONSE STRUCTURE

1. TOP-LEVEL FIELDS:
   • results: list
   • query: str
   • follow_up_questions: NoneType
   • answer: NoneType
   • images: list
   • results_count: int

2. RESULT OBJECT STRUCTURE:
   • url: https://www.timeout.com/newyork/events-calendar/ap...
   • title: NYC events in April 2025 - Time Out
   • content: The best NYC events in April include much-needed o...
   • score: 0.586
   • published_date: None
   • author: None
   • raw_content: None

3. WHAT WE EXTRACT FROM EACH RESULT:
   • url → candidate['url']
   • title → candidate['title']
   • content → candidate['snippet'] (truncated to 500 chars)
   • score → candidate['score']
   • published_date → candidate['published_date']


### 9.2 What Tavily API Response Looks Like

In [ ]:
# Let's see what parameters would be sent to Tavily API
# WITHOUT making an actual API call

# First, get a user profile from test data
profile_agent = ProfilePlannerAgent()
user_profile = profile_agent.build_user_profile()

# Create retriever agent
retriever = RetrieverAgent()

# Build the search query that would be sent
test_query = "Find cultural events and protests in NYC"
test_keywords = user_profile["keywords"][:5]
test_domains = user_profile["allowlist_domains"][:5]

# Build the full query string
full_query = retriever.build_search_query(
    base_query=test_query,
    keywords=test_keywords,
    location="NYC",
    date_range="next 14 days"
)

# Show what would be sent to Tavily
print("=" * 60)
print("TAVILY API REQUEST PARAMETERS (not actually sent)")
print("=" * 60)

tavily_request = {
    "query": full_query,
    "max_results": 15,
    "search_depth": "basic",
    "include_raw_content": False,
    "include_images": False,
    "include_domains": test_domains
}

print("\n1. QUERY STRING:")
print(f"   '{tavily_request['query']}'")

print("\n2. INCLUDE_DOMAINS (top 5):")
for domain in tavily_request["include_domains"]:
    print(f"   • {domain}")

print("\n3. OTHER PARAMETERS:")
print(f"   • max_results: {tavily_request['max_results']}")
print(f"   • search_depth: {tavily_request['search_depth']}")
print(f"   • include_raw_content: {tavily_request['include_raw_content']}")
print(f"   • include_images: {tavily_request['include_images']}")

print("\n4. ESTIMATED COST:")
print(f"   • 1 credit for basic search")

### 9.1 What Gets Sent to Tavily API

In [ ]:
# Import the Retriever agent to inspect it
from agents.retriever import RetrieverAgent
from agents.profile_planner import ProfilePlannerAgent
from datetime import datetime, timedelta

print("✅ Retriever agent imported for inspection")

## 9. Retriever Agent - Tavily API Inspection

In [6]:
# View the user profile that was built
print("User Profile Built by Profile & Planner:")
print("=" * 50)

profile = result['user_profile']

print("\nAllowed Domains:")
for domain in profile['allowlist_domains']:
    print(f"  • {domain}")

print("\nKeywords:")
for keyword in profile['keywords']:
    print(f"  • {keyword}")

print("\nInterest Areas:")
for interest in profile['interest_areas']:
    print(f"  • {interest}")

print("\nCredentials:")
for cred in profile['credentials']:
    print(f"  • {cred}")

User Profile Built by Profile & Planner:

Allowed Domains:
  • cnn.com
  • bbc.com
  • nytimes.com
  • un.org
  • whitehouse.gov
  • instagram.com
  • twitter.com

Keywords:
  • protest
  • cultural
  • political
  • parade
  • festival

Interest Areas:
  • protests
  • cultural events
  • political events

Credentials:
  • press_card


## 5. State Structure

In [7]:
# Examine the complete state structure
print("Workflow State Structure:")
print("=" * 50)

for key in result.keys():
    value = result[key]
    if isinstance(value, list):
        print(f"\n{key}: (list with {len(value)} items)")
        if len(value) > 0 and key != 'logs':  # Show first item except for logs
            print(f"  First item type: {type(value[0]).__name__}")
    elif isinstance(value, dict):
        print(f"\n{key}: (dict with {len(value)} keys)")
        print(f"  Keys: {list(value.keys())}")
    else:
        print(f"\n{key}: {value}")

Workflow State Structure:

query_spec: (dict with 6 keys)
  Keys: ['text', 'city', 'date_from', 'date_to', 'model', 'version']

user_profile: (dict with 5 keys)
  Keys: ['allowlist_domains', 'keywords', 'prior_feedback', 'interest_areas', 'credentials']

candidates: (list with 3 items)
  First item type: dict

extracted: (list with 3 items)
  First item type: dict

top10: (list with 3 items)
  First item type: dict

decision: (dict with 2 keys)
  Keys: ['action', 'notes']

cycle_count: 2

total_cost: 0.09

errors: (list with 0 items)

logs: (list with 17 items)


## 6. Cycle Behavior

In [8]:
# Analyze the cycle behavior
print("Cycle Analysis:")
print("=" * 50)

print(f"\nTotal cycles: {result['cycle_count']}")
print(f"Final decision: {result['decision']['action']}")
print(f"Decision notes: {result['decision']['notes']}")

# Count how many times each agent was called
agent_calls = {}
for log in result['logs']:
    agent = log.split(':')[0]
    agent_calls[agent] = agent_calls.get(agent, 0) + 1

print("\nAgent Execution Count:")
for agent, count in agent_calls.items():
    print(f"  • {agent}: {count} times")

Cycle Analysis:

Total cycles: 2
Final decision: accept
Decision notes: Sufficient quality results

Agent Execution Count:
  • Profile & Planner: 5 times
  • Retriever: 3 times
  • Extractor: 3 times
  • Recommender: 6 times


## 7. Test Different Queries

In [9]:
# Test with different query types
test_queries = [
    "Political events at UN headquarters",
    "Fashion and cultural events next week",
    "Protests and demonstrations in Bryant Park"
]

print("Testing Different Queries:")
print("=" * 50)

for query in test_queries:
    result = run_workflow(query, city="NYC", days_ahead=30)
    print(f"\nQuery: {query}")
    print(f"  Results: {len(result['top10'])} events")
    print(f"  Cycles: {result['cycle_count']}")
    print(f"  Cost: ${result['total_cost']:.4f}")
    if result['top10']:
        print(f"  Top result: {result['top10'][0]['title']}")

Testing Different Queries:

Query: Political events at UN headquarters
  Results: 3 events
  Cycles: 2
  Cost: $0.0900
  Top result: Climate Protest at Bryant Park

Query: Fashion and cultural events next week
  Results: 3 events
  Cycles: 2
  Cost: $0.0900
  Top result: Climate Protest at Bryant Park

Query: Protests and demonstrations in Bryant Park
  Results: 3 events
  Cycles: 2
  Cost: $0.0900
  Top result: Climate Protest at Bryant Park


## 8. Workflow Components

In [10]:
# Show the workflow components
print("Workflow Components:")
print("=" * 50)

print("\n1. STATE TYPES:")
print("   • QuerySpec: User's search parameters")
print("   • UserProfile: Domains, keywords, interests")
print("   • Candidates: Raw search results")
print("   • ExtractedEvent: Normalized event data")
print("   • RecommendedEvent: Scored events with rationales")
print("   • Decision: Gate decision (revise/accept)")

print("\n2. AGENT NODES:")
print("   • profile_planner_node: Builds search profile")
print("   • retriever_node: Searches for events (will use Tavily)")
print("   • extractor_node: Extracts event details (will use Tavily)")
print("   • recommender_gate_node: Scores and gates results")

print("\n3. FLOW:")
print("   Profile → Retriever → Extractor → Recommender/Gate")
print("   ↑                                              ↓")
print("   ←← (cycle if needed, max 2 times) ←←←←←←←←←←←←")

Workflow Components:

1. STATE TYPES:
   • QuerySpec: User's search parameters
   • UserProfile: Domains, keywords, interests
   • Candidates: Raw search results
   • ExtractedEvent: Normalized event data
   • RecommendedEvent: Scored events with rationales
   • Decision: Gate decision (revise/accept)

2. AGENT NODES:
   • profile_planner_node: Builds search profile
   • retriever_node: Searches for events (will use Tavily)
   • extractor_node: Extracts event details (will use Tavily)
   • recommender_gate_node: Scores and gates results

3. FLOW:
   Profile → Retriever → Extractor → Recommender/Gate
   ↑                                              ↓
   ←← (cycle if needed, max 2 times) ←←←←←←←←←←←←


## Summary

In [ ]:
print("=" * 60)
print("  LangGraph Workflow Status")
print("=" * 60)

print("\n✅ COMPLETED:")
print("   • State management system")
print("   • Agent node structure")
print("   • Linear workflow flow")
print("   • Cycle capability (with max limit)")
print("   • Cost tracking")
print("   • Logging system")

print("\n⏳ TODO (Next Steps):")
print("   • Profile & Planner: Parse real CSV data")
print("   • Retriever: Integrate Tavily search API")
print("   • Extractor: Use Tavily extract API")
print("   • Recommender: Improve scoring algorithm")

print("\n📍 Current Status:")
print("   The workflow skeleton is functional with mock data.")
print("   Ready to integrate real agents one by one.")